In [1]:
import arcpy
import random 
import pandas
from scipy.stats import pearsonr
import numpy as np
import os

In [2]:
GDB = r"C:\Users\jik720\Documents\ArcGIS\Projects\CensusRandom\CensusRandom.gdb"
arcpy.env.workspace = GDB
arcpy.env.overwriteOutput = 1

In [3]:
bg_fc = GDB + r'\blockgroups'
tracts_fc = GDB + r'\tracts'
counties_fc = GDB + r'\counties'
states_fc = GDB + r'\states'
indiv_fc = GDB + r'\random1'
joinFeatures = indiv_fc

$\color{blue}{\text{Big Loop}}$

In [4]:
for i in range(0,10):
    arcpy.CalculateField_management(indiv_fc, 'random1', "random.normalvariate(0,1)", 'PYTHON3', 'import random')
    arcpy.CalculateField_management(indiv_fc, 'random2', "random.normalvariate(0,1)", 'PYTHON3', 'import random')
    
    for targetFeatures in [bg_fc, tracts_fc, counties_fc, states_fc]:
        outfc = targetFeatures + '_joined'
        out_txt = "C:\\Users\\jik720\\Desktop\\" + targetFeatures[72:] + ".txt"
        print("working on " + outfc)
    
        fieldmappings = arcpy.FieldMappings()
        fieldmappings.addTable(targetFeatures)
        fieldmappings.addTable(joinFeatures)
    
        random1FieldIndex = fieldmappings.findFieldMapIndex("random1")
        fieldmap1 = fieldmappings.getFieldMap(random1FieldIndex)
        random2FieldIndex = fieldmappings.findFieldMapIndex("random2")
        fieldmap2 = fieldmappings.getFieldMap(random2FieldIndex)
    
        field1 = fieldmap1.outputField
        field2 = fieldmap2.outputField
    
        field1.name = "mean_rand1"
        field2.name = 'mean_rand2'
        fieldmap1.outputField = field1
        fieldmap2.outputField = field2
    
        fieldmap1.mergeRule = "mean"
        fieldmap2.mergeRule = "mean"
        fieldmappings.replaceFieldMap(random1FieldIndex, fieldmap1)
        fieldmappings.replaceFieldMap(random2FieldIndex, fieldmap2)
    
        arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings, 'INTERSECT')
        arcpy.stats.ExportXYv(outfc, ['mean_rand1', 'mean_rand2'], "COMMA", out_txt, 'ADD_FIELD_NAMES')
    
    print("writing")
    arcpy.stats.ExportXYv(indiv_fc, ['random1', 'random2'], "COMMA", r'C:\Users\jik720\Desktop\indiv2.txt', 'ADD_FIELD_NAMES')
    
    df_states = pandas.read_csv(r'C:\Users\jik720\Desktop\states.txt')
    df_counties = pandas.read_csv(r'C:\Users\jik720\Desktop\counties.txt')
    df_tracts = pandas.read_csv(r'C:\Users\jik720\Desktop\tracts.txt')
    df_blockgroups = pandas.read_csv(r'C:\Users\jik720\Desktop\blockgroups.txt')
    df_indiv = pandas.read_csv(r'C:\Users\jik720\Desktop\indiv2.txt')
    df_indiv.rename(columns={'RANDOM1':'MEAN_RAND1'}, inplace=True)
    df_indiv.rename(columns={'RANDOM2':'MEAN_RAND2'}, inplace=True)
    
    morans_list = []
    for fc in [states_fc, counties_fc, tracts_fc, bg_fc]:
        out_fc = fc + '_joined'
        moransI = arcpy.SpatialAutocorrelation_stats(out_fc, "MEAN_RAND1","NO_REPORT", "CONTIGUITY_EDGES_CORNERS", "EUCLIDEAN_DISTANCE", "ROW")
        moransI2 = arcpy.SpatialAutocorrelation_stats(out_fc, "MEAN_RAND2","NO_REPORT", "CONTIGUITY_EDGES_CORNERS", "EUCLIDEAN_DISTANCE", "ROW")
        morans_list.append([moransI[0], moransI2[0]])
    
    R_list = []
    std_list = []
    
    for df in [df_states, df_counties, df_tracts, df_blockgroups, df_indiv]:
        R_list.append(round(pearsonr(df.MEAN_RAND1, df.MEAN_RAND2)[0], 5))
        std_list.append([round(df.MEAN_RAND1.std(),5), round(df.MEAN_RAND2.std(),5)])
    
    level_list  =  ['state', 'county', 'tract', 'blockgroup', 'individual']
    out_list = list(zip(level_list, R_list, std_list, morans_list))
    print(out_list)
    
    results_txt = r'C:\Users\jik720\Desktop\results.txt'
    with open(results_txt, "a") as o_file:
        o_file.write('-----\n')
        for item in out_list:
            o_file.write(str(item) + '\n')
            
    os.remove(r'C:\Users\jik720\Desktop\indiv2.txt')

working on C:\Users\jik720\Documents\ArcGIS\Projects\CensusRandom\CensusRandom.gdb\blockgroups_joined
working on C:\Users\jik720\Documents\ArcGIS\Projects\CensusRandom\CensusRandom.gdb\tracts_joined
working on C:\Users\jik720\Documents\ArcGIS\Projects\CensusRandom\CensusRandom.gdb\counties_joined
working on C:\Users\jik720\Documents\ArcGIS\Projects\CensusRandom\CensusRandom.gdb\states_joined
writing
[('state', -0.19956, [0.00221, 0.0022], ['0.038992', '-0.158125']), ('county', -0.01925, [0.03085, 0.03049], ['-0.007649', '-0.025517']), ('tract', -0.00213, [0.06373, 0.06345], ['0.002464', '0.000727']), ('blockgroup', -0.00135, [0.10022, 0.10004], ['0.000362', '-0.000841'])]
working on C:\Users\jik720\Documents\ArcGIS\Projects\CensusRandom\CensusRandom.gdb\blockgroups_joined
working on C:\Users\jik720\Documents\ArcGIS\Projects\CensusRandom\CensusRandom.gdb\tracts_joined
working on C:\Users\jik720\Documents\ArcGIS\Projects\CensusRandom\CensusRandom.gdb\counties_joined
working on C:\Users\j